In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import tensorflow as tf

Using TensorFlow backend.


In [2]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'cats-dogs/train/'
validation_data_dir = 'cats-dogs/test/'



nb_train_samples = 75
nb_validation_samples = 50
#epochs = 50
#batch_size = 16
epochs = 10
batch_size = 10

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [3]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])



train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)


test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=100)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.

Epoch 1/10
100/100 [==============================] - 45s 445ms/step - loss: 0.7305 - accuracy: 0.5300 - val_loss: 0.6864 - val_accuracy: 0.5630
Epoch 2/10
100/100 [==============================] - 43s 429ms/step - loss: 0.7186 - accuracy: 0.5340 - val_loss: 0.5884 - val_accuracy: 0.5080
Epoch 3/10
100/100 [==============================] - 44s 436ms/step - loss: 0.6905 - accuracy: 0.5820 - val_loss: 0.6011 - val_accuracy: 0.6260
Epoch 4/10
100/100 [==============================] - 44s 439ms/step - loss: 0.6522 - accuracy: 0.6350 - val_loss: 0.6384 - val_accuracy: 0.6740
Epoch 5/10
100/100 [==============================] - 44s 438ms/step - loss: 0.6354 - accuracy: 0.6440 - val_loss: 0.7916 - val_accuracy: 0.6400
Epoch 6/10
100

In [4]:
#model.fit_generator(
#    train_generator,
 #   steps_per_epoch=nb_train_samples // batch_size,
  #  epochs=epochs,
   # validation_data=validation_generator,
    #validation_steps=nb_validation_samples // batch_size)

In [5]:
# Save tf.keras model in HDF5 format.
keras_file = "7_try.h5"
model.save(keras_file)

In [6]:
# Convert to TensorFlow Lite model.

converter = tf.lite.TFLiteConverter.from_keras_model_file(keras_file)
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 10 variables.
INFO:tensorflow:Converted 10 variables to const ops.


4852748